## Day 25 Lecture 1 Assignment

In this assignment, we will evaluate the performance of the model we built yesterday on the Chicago traffic crash data. We will also perform hyperparameter tuning and evaluate a final model using additional metrics (e.g. AUC-ROC, precision, recall, etc.)

In [7]:
%reload_ext nb_black
import numpy as np
import pandas as pd

import statsmodels.api as sm

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
from sklearn.metrics import (
    log_loss,
    classification_report,
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    make_scorer,
)
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Since we will be building on the model we built in the last assignment, we will need to redo all of the data preparation steps up to the point of model building. These steps include creating the response, missing value imputation, and one-hot encoding our selected categorical variables. The quickest way to get going would be to open last week's assignment, make a copy, and build on it from there.

In [2]:
crash_data = pd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/traffic_crashes_chicago.csv"
)
crash_data.head()

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN
0,JC334993,7/4/2019 22:33,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JC370822,7/30/2019 10:22,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JC387098,8/10/2019 17:00,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JC395195,8/16/2019 16:53,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,...,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,JC396604,8/17/2019 16:04,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,PARKING LOT,NaN,...,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0


<IPython.core.display.Javascript object>

In [3]:
crash_data["is_over_1500"] = crash_data["DAMAGE"] == "OVER $1,500"
crash_data["is_over_1500"] = crash_data["is_over_1500"].astype(int)

crash_data = crash_data.drop(columns=["DAMAGE"])

crash_data["is_over_1500"].value_counts(normalize=True)

1    0.563418
0    0.436582
Name: is_over_1500, dtype: float64

<IPython.core.display.Javascript object>

In [14]:
# Dropping all columns with more than 5% missing data
is_na_perc = crash_data.isna().mean()
drop_cols = is_na_perc[is_na_perc > 0.05].index
crash_data = crash_data.drop(columns=drop_cols)

# Imputing the median for numeric columns with less than 5% missing data (except for STREET_NO)
col_medians = crash_data.drop(columns="STREET_NO").median()
crash_data = crash_data.fillna(col_medians)

# Dropping rows with missing data for categorical columns that have less than 5% missing data
crash_data = crash_data.dropna()

x_cols = [
    "POSTED_SPEED_LIMIT",
    "WEATHER_CONDITION",
    "INJURIES_TOTAL",
    "FIRST_CRASH_TYPE",
]
X = crash_data[x_cols]
y = crash_data["is_over_1500"]

drop_cats = ["WEATHER_CONDITION_CLEAR", "FIRST_CRASH_TYPE_REAR END"]
X_dummies = pd.get_dummies(X)
X_dummies = X_dummies.drop(columns=drop_cats)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X_dummies, y, test_size=0.2, random_state=42
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Statsmodels' implementation of logistic has certain advantages over scikit-learn's, such as clean, easy to read model summary output and statistical inference values (e.g. p-values). However, scikit-learn is preferable for model evaluation, so we will switch to the scikit-learn implementation for this exercise. 

Run logistic regression on the training set and use the resulting model to make predictions on the test set. Calculate the train and test error using logarithmic loss. How do they compare to each other?

In [16]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
model.score(X_train, y_train)

0.5831109088024939

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
model.score(X_test, y_test)

0.5857483757948605

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Next, evaluate the performance of the same model using 10-fold CV. Use the training data and labels, and print out the mean log loss for each of the 10 CV folds, as well as the overall CV-estimated test error. How do the estimates from the individual folds compare to the result from our previous single holdout set? How much variability in the estimated test error do you see across the 10 folds?

Note: scikit-learn's *cross_val_score* function provides a simple, one-line method for doing this. However, be careful - the default score returned by this function may not be log loss!

In [28]:
#loss = inverse of accuracy
cross_val_score(model, X_train, y_train, cv=10, scoring="neg_log_loss")

array([-0.65759163, -0.65707865, -0.65947468, -0.65860472, -0.65937332,
       -0.65916443, -0.65699471, -0.65980692, -0.65804188, -0.65693655])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
cross_val_score(model, X_train, y_train, cv=10)

array([0.58015794, 0.58319252, 0.58184765, 0.58205455, 0.58333046,
       0.58036484, 0.58635078, 0.58245396, 0.58221257, 0.58814401])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
from sklearn.metrics import log_loss

x = model.predict_proba(X_test)
x[:, 1]
log_loss(y_test, x[:, 1])

0.6570250587956836

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
y_test

277527    0
236957    1
179408    0
110770    0
323859    1
         ..
268535    1
37314     0
355393    0
271312    0
76279     1
Name: is_over_1500, Length: 72497, dtype: int32

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Scikit-learn's logistic regression function has a built-in regularization parameter, C (the larger the value of C, the smaller the degree of regularization). Use a loop with values of C, and find the optimum C value by taking the highest average cross validation score.

Re-train a logistic regression model using the best value of C identified by 10-fold CV on the training data and labels. Afterwards, do the following:

- Determine the precision, recall, and F1-score of our model using a cutoff/threshold of 0.5 (hint: scikit-learn's *classification_report* function may be helpful)
- Plot or otherwise generate a confusion matrix
- Plot the ROC curve for our logistic regression model

Note: the performance of our simple logistic regression model with just four features will not be very good, but this is not entirely unexpected. There are many other features that can be incorporated into the model to improve its performance; feel free to experiment!